In [26]:
from PIL import Image
import os
import numpy as np
from pypdf import PdfMerger, PdfReader, PdfWriter

In [22]:
import io
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

Utility script to make serveral figures in one pdf, which I can share more easily.

# Image Report

In [4]:
current_dir = globals()['_dh'][0]
for quality in ["hires", "fullres"]: 
    input_dir = current_dir / ".." / "data" / ("image_features_" + quality)
    out_pdf = current_dir / ".." / "out" / ("image_features_eval_" + quality + ".pdf")

    files = [f for f in os.listdir(input_dir) if not f.startswith(".") and f.endswith(".png")]
    samples = np.unique(np.array([f.split("_")[0] for f in files]))

    images = []
    for sample in samples:
        files_oi = [f for f in files if f.startswith(sample)]
        for img in files_oi:
            png = Image.open(input_dir / img)
            png.load() # required for png.split()
            background = Image.new("RGB", png.size, (255, 255, 255))
            background.paste(png, mask=png.split()[3]) # 3 is the alpha channel
            images.append(background)

    images[0].save(
        out_pdf, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:]
    )

# QC Report

In [2]:
current_dir = globals()['_dh'][0]
for method in ["cellbender", "cellranger"]:
    input_dir = current_dir / ".." / "out" / (method + "_qc")
    out_pdf = current_dir / ".." / "out" / (method + "_qc_out.pdf")

    images = []
    files = [f for f in os.listdir(input_dir) if not f.startswith(".") and f.endswith(".png")]
    for img in files:
        png = Image.open(input_dir / img)
        png.load() # required for png.split()
        background = Image.new("RGB", png.size, (255, 255, 255))
        background.paste(png, mask=png.split()[3]) # 3 is the alpha channel
        images.append(background)

    images[0].save(
        out_pdf, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:]
    )

# CellBender Report

In [ ]:
packet = io.BytesIO()
can = canvas.Canvas(packet, pagesize=letter)
can.drawString(10, 100, "Hello world")
can.save()
packet.seek(0)

In [31]:
current_dir = globals()['_dh'][0]
input_dir = current_dir / ".." / "data" / "prc" / "sc" / "cellbender"
output_pdf = current_dir / ".." / "out" / "cellbender_report.pdf"
merger = PdfMerger()
samples = os.listdir(input_dir)
for sample in samples:
    pdf = input_dir / sample / "cell_bender_matrix.pdf"
    pdf_out = input_dir / sample / "cell_bender_matrix_s.pdf"

    # add sample name to pdf
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=letter)
    can.drawString(10, 800, sample)
    can.save()
    packet.seek(0)

    new_pdf = PdfReader(packet)
    existing_pdf = PdfReader(open(pdf, "rb"))
    output = PdfWriter()
    page = existing_pdf.pages[0]
    page.merge_page(new_pdf.pages[0])
    output.add_page(page)
    output_stream = open(pdf_out, "wb")
    output.write(output_stream)
    output_stream.close()

    merger.append(pdf_out)

merger.write(output_pdf)
merger.close()